In [4]:
import os

os.environ['CUDA_HOME'] = '/home/yalandur/research/cuda121/'
os.environ['CUDA_TOOLKIT_ROOT_DIR'] = os.environ['CUDA_HOME']
os.environ['LD_LIBRARY_PATH'] = f"{os.environ['CUDA_HOME']}/extras/CUPTI/lib64"
os.environ['LIBRARY_PATH'] = f"{os.environ['CUDA_HOME']}/lib64"
os.environ['LD_LIBRARY_PATH'] = os.environ['CUDA_HOME'] + '/lib64'
os.environ['CFLAGS'] = f"-I{os.environ['CUDA_HOME']}/include {os.environ.get('CFLAGS', '')}"

In [5]:
from run import *

In [6]:
assert (torch.cuda.is_available())
device = f"cuda:{0}"

opt = OmegaConf.load("configs/image.yaml")
scene = get_scene(opt.scene.type, opt.scene.attributes)

In [ ]:

out_dir = Path(opt.scene.save_dir)
print(f"Saving to {out_dir}")
out_dir.mkdir(exist_ok=True, parents=True)
(out_dir / "crops").mkdir(exist_ok=True)
(out_dir / "object_space").mkdir(exist_ok=True)
(out_dir / "reconstruction").mkdir(exist_ok=True)

OmegaConf.save(config=opt, f=out_dir / "config.yaml")
scene.image_pil.save(out_dir / 'input.png')

K_img, pose = get_camera(scene, opt.run.calibration)
depth_map = get_depth(scene, opt.run.depth, K_img, out_dir, device)
masks, object_ids, background_ids, instance_labels = get_components_masks(scene, opt.run.segmentation, out_dir, device)

pts3d = depth_to_points(depth_map[None], K_img)
trimesh.PointCloud(pts3d.reshape(-1, 3), scene.image_np.reshape(-1, 3)).export(out_dir / 'depth_scene.ply')
cam_params = {
    'K': K_img.tolist(),
    'c2w': pose.tolist(),
    'W': scene.image_pil.width,
    'H': scene.image_pil.height,
}
with open(out_dir / 'cam_params.json', 'w') as fp:
    json.dump(cam_params, fp)

if not opt.run.only_foreground:
    background_mesh = get_background_mesh(
        scene.image_np, pts3d, np.any(masks[background_ids], axis=0),
        K_img, depth_map, device)
    background_mesh.apply_transform(pose)
    background_mesh.export(out_dir / 'reconstruction' / 'background.ply')
else:
    background_mesh = None

K_crop = get_crop_calibration(args.crop_size)

Saving to ../out/demo_1
Using {'intrinsics': 'perspectivefields', 'pose': None} for camera calibration


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 128 n 100 k 64 mat1_ld 64 mat2_ld 64 result_ld 128 abcType 0 computeType 68 scaleType 0

In [7]:
from perspective2d import PerspectiveFields
from perspective2d.utils.utils import general_vfov_to_focal


/home/yalandur/research/Gen3DSR/.venv/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [8]:
img_bgr = scene.image_np[..., ::-1]
# resize image to 320x320
img_bgr = cv2.resize(img_bgr, (320, 320), interpolation=cv2.INTER_LINEAR)
pf_model = PerspectiveFields('Paramnet-360Cities-edina-uncentered').eval().cuda()
pred = pf_model.inference(img_bgr=img_bgr)

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 128 n 100 k 64 mat1_ld 64 mat2_ld 64 result_ld 128 abcType 0 computeType 68 scaleType 0

In [3]:
import torch

# Create two random matrices
a = torch.randn(1024, 1024, device='cuda')
b = torch.randn(1024, 1024, device='cuda')

# Perform a matrix multiplication
c = torch.matmul(a, b)
print("Matrix multiplication successful!")

Matrix multiplication successful!


In [9]:
# torch version
print(torch.__version__)

2.2.2+cu121
